In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')

In [4]:
data.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [5]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
#encode categorical variables
label_enc_gender=LabelEncoder()
data['Gender']=label_enc_gender.fit_transform(data['Gender'])

In [8]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [9]:
#one hot encode the geography

one_hot_enc_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=one_hot_enc_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_enc_geo.get_feature_names_out(['Geography']))


In [10]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
#combine
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [12]:
#split the data 
x=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [13]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [14]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [15]:
with open('label.pkl','wb') as file:
    pickle.dump(label_enc_gender,file)

with open('onehot.pkl','wb') as file:
    pickle.dump(one_hot_enc_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN REGRESSION PBM STATEMENT

In [16]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [20]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),#HL2
    Dense(1)
])


In [23]:
#complie the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [24]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
#set up the tensorboard
import datetime
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir='logs/fit'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [28]:
#set up early stopping
early_stoppin_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [29]:
##train the model
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,
                  callbacks=[tensorflow_callback,early_stoppin_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100394.8984 - mae: 100394.8984 - val_loss: 98580.9062 - val_mae: 98580.9062
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99850.4375 - mae: 99850.4375 - val_loss: 97465.4219 - val_mae: 97465.4219
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 97878.2891 - mae: 97878.2891 - val_loss: 94549.0000 - val_mae: 94549.0000
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 93906.4922 - mae: 93906.4922 - val_loss: 89536.7422 - val_mae: 89536.7422
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 87895.6094 - mae: 87895.6094 - val_loss: 82683.7344 - val_mae: 82683.7344
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 80319.3125 - mae: 80319.3125 - val_loss: 74814.0000 - val_mae: 74814.0000
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 72237.7891 - mae: 72237.7891 - val_loss: 67240.6250 - val_mae: 67240.6250
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 

In [30]:
model.save('model_regression.h5')

In [31]:
%load_ext tensorboard

In [32]:
%tensorboard --logdir logs/fit20260120-234240

In [34]:
#evaluate model on test data
test_loss,test_mae=model.evaluate(x_test,y_test)
print(f'test mae: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50289.2617 - mae: 50289.2617
test mae: 50289.26171875
